In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from catboost import CatBoostRegressor

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

## Подготовка данных

In [4]:
#создаем метку, для различия train и test
train['mode'] = 'train'
test['mode'] = 'test'

#соединяем train и test
data = pd.concat([train, test])

In [5]:
data = data.sort_values(['store_id', 'product_id', 'date']).reset_index(drop=True)
data

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,pressure,sales,mode
0,1,2021-07-29,Москва,1,1,1,4.79,"переменная облачность, небольшой дождь",61.9375,23.1875,741.0000,26.0,train
1,2,2021-07-30,Москва,1,1,1,4.79,"переменная облачность, небольшой дождь",70.2500,22.1875,740.3125,37.0,train
2,3,2021-07-31,Москва,1,1,1,4.79,переменная облачность,52.6250,21.8125,741.6250,25.0,train
3,4,2021-08-01,Москва,1,1,1,4.79,"облачно, небольшой дождь",87.4375,20.0625,743.3125,26.0,train
4,5,2021-08-02,Москва,1,1,1,4.79,переменная облачность,66.1875,23.4375,739.6250,22.0,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...
691507,691508,2022-02-16,Воронеж,164,1,35,2.48,переменная облачность,91.8125,-2.5625,751.0000,NaN,test
691508,691509,2022-02-17,Воронеж,164,1,35,2.48,"облачно, небольшой дождь",93.5000,2.0625,740.0000,NaN,test
691509,691510,2022-02-18,Воронеж,164,1,35,2.48,облачно,96.5625,2.0625,736.0000,NaN,test
691510,691511,2022-02-19,Воронеж,164,1,35,2.48,переменная облачность,89.9375,2.7500,743.0000,NaN,test


In [6]:
data['date'] = pd.to_datetime(data['date'])


#день недели
data['day'] = data['date'].dt.weekday
data['month'] = data['date'].dt.month

In [7]:
#группируем по магазину и товару
group = data.groupby(['store_id', 'product_id'])
#генеригуем лаги
for i in range(7, 21):
    data[f'lag_{i}'] = group['sales'].shift(i)
    

    
#выкидываем нача
data.fillna(value=0, inplace=True)

In [8]:
#week_index = data['date'].dt.to_period('W', )
#data = pd.concat([data, week_index.rename('week_index')], axis=1)
#data.groupby(['week_index', 'store_id', 'product_id'])['sales'].mean()

In [9]:
#one-hot encoding для weather_desc
data = pd.concat((data, pd.get_dummies(data['weather_desc'], prefix='wd')), axis=1)

In [10]:
#label encoding для city_name
city_name_id = dict(zip(data['city_name'].unique(), range(len(data['city_name'].unique()))))
data['city_name_id'] = data['city_name'].apply(lambda x: city_name_id[x])

In [11]:
data

,id,date,city_name,store_id,category_id,product_id,price,weather_desc,humidity,temperature,...,"wd_облачно, небольшой снег",wd_осадки,wd_переменная облачность,"wd_переменная облачность, дождь","wd_переменная облачность, небольшие осадки","wd_переменная облачность, небольшой дождь","wd_переменная облачность, небольшой снег",wd_снег,wd_ясно,city_name_id
0,1,2021-07-29,Москва,1,1,1,4.79,"переменная облачность, небольшой дождь",61.9375,23.1875,...,0,0,0,0,0,1,0,0,0,0
1,2,2021-07-30,Москва,1,1,1,4.79,"переменная облачность, небольшой дождь",70.2500,22.1875,...,0,0,0,0,0,1,0,0,0,0
2,3,2021-07-31,Москва,1,1,1,4.79,переменная облачность,52.6250,21.8125,...,0,0,1,0,0,0,0,0,0,0
3,4,2021-08-01,Москва,1,1,1,4.79,"облачно, небольшой дождь",87.4375,20.0625,...,0,0,0,0,0,0,0,0,0,0
4,5,2021-08-02,Москва,1,1,1,4.79,переменная облачность,66.1875,23.4375,...,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691507,691508,2022-02-16,Воронеж,164,1,35,2.48,переменная облачность,91.8125,-2.5625,...,0,0,1,0,0,0,0,0,0,7
691508,691509,2022-02-17,Воронеж,164,1,35,2.48,"облачно, небольшой дождь",93.5000,2.0625,...,0,0,0,0,0,0,0,0,0,7
691509,691510,2022-02-18,Воронеж,164,1,35,2.48,облачно,96.5625,2.0625,...,0,0,0,0,0,0,0,0,0,7
691510,691511,2022-02-19,Воронеж,164,1,35,2.48,переменная облачность,89.9375,2.7500,...,0,0,1,0,0,0,0,0,0,7


## Модель

In [12]:
drop_columns = ['id', 'date', 'weather_desc', 'temperature', 'pressure', 'humidity', 'sales', 'city_name', 'mode', 'price']
features = list(set(data.columns) - set(drop_columns))

In [13]:
features

['month',
 'wd_облачно, небольшие осадки',
 'wd_облачно, без существенных осадков',
 'lag_14',
 'lag_20',
 'wd_переменная облачность, дождь',
 'lag_17',
 'wd_снег',
 'city_name_id',
 'lag_13',
 'store_id',
 'wd_переменная облачность',
 'wd_облачно, небольшой снег',
 'wd_ясно',
 'lag_12',
 'wd_переменная облачность, небольшой дождь',
 'lag_10',
 'wd_облачно, небольшой дождь',
 'wd_осадки',
 'day',
 'lag_7',
 'wd_облачно',
 'lag_8',
 'wd_переменная облачность, небольшой снег',
 'wd_переменная облачность, небольшие осадки',
 'product_id',
 'lag_16',
 'lag_19',
 'lag_11',
 'wd_метель',
 'wd_дождь',
 'lag_15',
 'wd_дождь, гроза',
 'category_id',
 'lag_9',
 'lag_18']

In [14]:
#Разбиваем датасет на train и test
train_df = data[(data['mode'] == 'train')].copy()
test_df = data[data['mode'] == 'test'].copy()

In [15]:
y_train = train_df['sales']
y_test = test_df['sales']

X_train = train_df[features]
X_test = test_df[features]

In [16]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [17]:
model = CatBoostRegressor(loss_function='MAE', silent=True, num_boost_round=200, max_depth=10)
model.fit(X_train, y_train)

In [18]:
predict = model.predict(X_test)

In [19]:
#проверим MAE на тренировочных данных(хотя это не является показателем качества)
train_predict = model.predict(X_train)
mean_absolute_error(y_train, train_predict)

3.7030385288202137

## Вывод в файл

In [20]:
output = pd.DataFrame({'id': test['id'],
                      'prediction': predict})

output.to_csv('prediction.csv', index=False)
print(output)

           id  prediction
0      666677   26.414861
1      666678   27.335358
2      666679   29.482825
3      666680   29.114416
4      666681   39.745336
...       ...         ...
24831  691508   13.450552
24832  691509   14.340162
24833  691510   12.882062
24834  691511   12.936055
24835  691512   14.789369

[24836 rows x 2 columns]
